# Machine Learning Recommendation Model
This model will be trained using data from user's ratings of a variety of anime
## Imports
Setting up the imports that will be needed for the model training

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

## Spark Session
Creating a spark session by creating a spark environment

In [4]:
SparkSession.builder \
            .appName('AnimeRec') \
            .config('spark.driver.memory', '4g') \
            .getOrCreate()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.